In [58]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import pickle
import gensim
import pyLDAvis
import pyLDAvis.gensim_models
import spacy
import pandas as pd
import nltk; nltk.download('stopwords')
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import re
import warnings
from pprint import pprint

[nltk_data] Downloading package stopwords to /Users/hieu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [59]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

df = pd.read_pickle("../data/pickle/final_wo_errors.pkl")
df = df.drop(1727)


In [60]:
df['nlp'] = df['description_aims']
df["nlp"] = df["nlp"].fillna("")

In [61]:
stop_words.extend(["united_kingdom", "public", "sector", "programme", "hmrc"])

In [62]:
def strip_newline(series):
    return [review.replace('\n','') for review in series]

df["nlp"] = strip_newline(df["nlp"])

In [63]:
# Create a dictionary of the characters to replace
replacements = {
    'Âs': ' ',
    '\xa0': ' ',
    '\n': ' ',
    '\r': ' ',
    '\x96': ' ',
    '\t': ' ',
    "\'s": ' ',
    "'s": ' ',
    '\x92': ' ',
    '\x95': ' ',
    '\x93': ' ',
    '\x94': ' ',
    '\x91': ' '
}

# Define a function to replace the unwanted characters
def clean_text(text):
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text

# Apply the function to the "nlp" column
df["nlp"] = df["nlp"].apply(clean_text)

In [64]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

words_tr = list(sent_to_words(df["nlp"]))

In [65]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

words_tr = remove_stopwords(words_tr)

In [66]:
def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    trigram = gensim.models.Phrases(bigram[words], min_count = tri_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    return bigram_mod, trigram_mod

bigram_tr, trigram_tr = bigrams(words_tr)

In [67]:
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])

In [68]:
def lemmatization(texts, allowed_postags=['NOUN','ADJ','VERB','ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


trigrams_tr = [trigram_tr[bigram_tr[review]] for review in words_tr]

lemma_lg = lemmatization(trigrams_tr)


In [79]:
bigram_tr

In [78]:
words_tr

[['pensions',
  'remedy',
  'created',
  'end',
  'age',
  'discrimination',
  'within',
  'civil',
  'service',
  'pension',
  'schemes',
  'also',
  'creating',
  'solutions',
  'remediate',
  'affected',
  'historic',
  'members',
  'scheme'],
 ['original',
  'commercial',
  'capability',
  'successfully',
  'established',
  'government',
  'commercial',
  'organisation',
  'gco',
  'single',
  'central',
  'employer',
  'several',
  'hundred',
  'commercial',
  'specialists',
  'grade',
  'central',
  'government',
  'departments',
  'civil',
  'service',
  'board',
  'endorsed',
  'proposals',
  'commercial',
  'capability',
  'building',
  'interventions',
  'extended',
  'grade',
  'commercial',
  'professionals',
  'within',
  'central',
  'government',
  'wider',
  'government',
  'bodies',
  'wgbs',
  'training',
  'accreditation',
  'developed',
  'delivered',
  'civil',
  'service',
  'contract',
  'management',
  'community',
  'commercial',
  'capability',
  'expansion',


In [77]:
trigrams_tr

[['pensions',
  'remedy',
  'created',
  'end',
  'age',
  'discrimination',
  'within',
  'civil_service',
  'pension_schemes',
  'also',
  'creating',
  'solutions',
  'remediate',
  'affected',
  'historic',
  'members',
  'scheme'],
 ['original',
  'commercial',
  'capability',
  'successfully',
  'established',
  'government',
  'commercial',
  'organisation',
  'gco',
  'single',
  'central',
  'employer',
  'several',
  'hundred',
  'commercial',
  'specialists',
  'grade',
  'central',
  'government_departments',
  'civil_service',
  'board',
  'endorsed',
  'proposals',
  'commercial',
  'capability',
  'building',
  'interventions',
  'extended',
  'grade',
  'commercial',
  'professionals',
  'within',
  'central',
  'government',
  'wider',
  'government',
  'bodies',
  'wgbs',
  'training',
  'accreditation',
  'developed',
  'delivered',
  'civil_service',
  'contract',
  'management',
  'community',
  'commercial',
  'capability',
  'expansion',
  'established',
  'impac

In [69]:
with open('../data/pickle/lemma_lg.pkl', 'wb') as f:
    pickle.dump(lemma_lg, f)

In [70]:
id2word_lg = gensim.corpora.Dictionary(lemma_lg)
id2word_lg.filter_extremes(no_below=10, no_above=0.35)
id2word_lg.compactify()
id2word_lg.save('train_dict_lg')
corpus_lg = [id2word_lg.doc2bow(text) for text in lemma_lg]

In [71]:
with open('../data/pickle/corpus_lg.pkl', 'wb') as f:
    pickle.dump(corpus_lg, f)

In [72]:
[(id2word_lg[id], freq) for id, freq in corpus_lg[21]]


[('address', 1),
 ('contribute', 1),
 ('include', 1),
 ('sustainable', 1),
 ('aims_deliver', 1),
 ('consumption', 1),
 ('environmental', 1),
 ('food', 1),
 ('home', 1),
 ('produce', 1),
 ('renew', 1),
 ('significantly', 1)]

In [73]:
from gensim.models import HdpModel
hdp = HdpModel(corpus_lg, id2word_lg, chunksize=10000)

In [74]:
len(hdp.print_topics())

20

In [75]:
hdp.print_topics(num_topics=20)


[(0,
  '0.005*charge + 0.005*previously + 0.004*century + 0.004*hold + 0.004*prevent + 0.004*asset + 0.004*wide + 0.004*well + 0.004*uncertainty + 0.004*large'),
 (1,
  '0.005*ict_contract + 0.005*continue + 0.005*track + 0.005*condition + 0.005*level + 0.005*digitally + 0.004*credit + 0.004*serious + 0.004*globally + 0.004*book'),
 (2,
  '0.007*start + 0.006*host + 0.005*big + 0.005*credit + 0.004*securely + 0.004*choice + 0.004*propose + 0.004*carrier + 0.004*refurbish + 0.004*signal'),
 (3,
  '0.007*paper + 0.006*deal + 0.005*personal + 0.005*voice + 0.005*storage + 0.005*innovate + 0.005*understand + 0.004*network + 0.004*productivity + 0.004*chief'),
 (4,
  '0.007*accommodation + 0.006*century + 0.006*together + 0.005*parent + 0.005*littoral_land + 0.005*pace + 0.005*bringing_together + 0.005*ict_contract + 0.005*ground + 0.005*prison_population'),
 (5,
  '0.005*operation + 0.005*play + 0.005*give + 0.005*continuation + 0.005*oversight + 0.005*support + 0.005*keep + 0.004*standard

In [76]:
words = ['This', 'is', 'an', 'example']
sentence = " ".join(words)
print(sentence)
# Output: 'This is an example'

This is an example
